In [1]:
from relationGraph import Relation, RelationGraph, MatrixOfRelationGraph
from base import load_source
from os.path import join
import numpy as np
from main import test_build_relation_graph_with_symertic_data, test_convert_graph_to_2D_matrix, test_get_matix_for_autoencoder

graph = test_build_relation_graph_with_symertic_data()
test_get_matix_for_autoencoder(graph)


-------------RelationGraph-------------
GO term	116
1	ann_T-(116, 1219)
1	ann-(1219, 116)
Experimental condition	282
1	expr_T-(282, 1219)
1	expr-(1219, 282)
Gene	1219
3	ann-(1219, 116), expr-(1219, 282), ppi-(1219, 1219)
2	ann_T-(116, 1219), expr_T-(282, 1219)

ann_T (116, 1219)
ann (1219, 116)
expr_T (282, 1219)
expr (1219, 282)
ppi (1219, 1219)
GO term: 12
Experimental condition: 28
Gene: 122
(162, 162)
[0. 0. 0. ... 0. 0. 0.]


array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [3.62695086, 5.36584517, 4.71983726, ..., 0.        , 0.        ,
        0.        ],
       [3.6707664 , 5.48442342, 4.73802135, ..., 0.        , 0.        ,
        0.        ],
       [3.66440744, 7.15474816, 6.39846681, ..., 0.        , 0.        ,
        0.        ]])

In [2]:
graph.display_objects()

-------------RelationGraph-------------
Experimental condition	282
1	expr_T-(282, 1219)
1	expr-(1219, 282)
GO term	116
1	ann_T-(116, 1219)
1	ann-(1219, 116)
Gene	1219
3	ann-(1219, 116), expr-(1219, 282), ppi-(1219, 1219)
2	ann_T-(116, 1219), expr_T-(282, 1219)



In [4]:
mrg = MatrixOfRelationGraph(graph=graph)
mrg.convert_to_2D_matrix()
data = mrg.density_data()
print(data.shape)
data = data.flatten()
print(data.shape)

ann (1219, 116)
expr (1219, 282)
ppi (1219, 1219)
ann_T (116, 1219)
expr_T (282, 1219)
Gene: 122
GO term: 12
Experimental condition: 28
(162, 162)
(26244,)


In [6]:
from tensorflow.keras.layers import Input, Dropout, Dense, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import mnist
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow import set_random_seed
import numpy as np
import os


def seedy(s):
    np.random.seed(s)
    set_random_seed(s)


class AutoEncoder:

    def __init__(self, encoding_dim=3, data=None):
        self.encoding_dim = encoding_dim
#         r = lambda: np.random.randint(1, 3)
#         self.x = np.array([[r(), r(), r()] for _ in range(1000)])
        self.x = np.array([data for _ in range(1000)])
        self.x_dim = self.x.shape[1]
        print(self.x)
        print(self.x_dim)

    def _encoder(self):
        inputs = Input(shape=(self.x[0].shape))
        encoded = Dense(self.encoding_dim, activation='relu')(inputs)
        model = Model(inputs, encoded)
        self.encoder = model
        return model

    def _decoder(self):
        inputs = Input(shape=(self.encoding_dim,))
        decoded = Dense(self.x_dim)(inputs)
        model = Model(inputs, decoded)
        self.decoder = model
        return model

    def encoder_decoder(self):
        ec = self._encoder()
        dc = self._decoder()

        inputs = Input(shape=self.x[0].shape)
        ec_out = ec(inputs)
        dc_out = dc(ec_out)
        model = Model(inputs, dc_out)

        self.model = model
        return model

    def fit(self, batch_size=100, epochs=300):
        self.model.compile(optimizer='sgd', loss='mse')
        log_dir = '/mag/logs/'
        callbacks = [TensorBoard(log_dir=log_dir, histogram_freq=0, write_graph=True, write_images=True)]

        self.model.fit(self.x, self.x,
                       epochs=epochs,
                       batch_size=batch_size,
                       callbacks=callbacks
                      )

    def save(self):
        if not os.path.exists(r'/mag/weigths'):
            os.mkdir(r'/mag/weigths')

        self.encoder.save(r'/mag/weigths/encoder_weigths.h5')
        self.decoder.save(r'/mag/weigths/decoder_weigths.h5')
        self.model.save(r'/mag/weigths/ae_weigths.h5')


if __name__ == '__main__':
    seedy(2)
    print(data.shape)
    ae = AutoEncoder(encoding_dim=2, data=data)
    ae.encoder_decoder()
    ae.fit(batch_size=50, epochs=300)
#     ae.save()

(26244,)
[[5.00661389 4.83041553 4.8573831  ... 0.79615493 4.66824827 4.0636104 ]
 [5.00661389 4.83041553 4.8573831  ... 0.79615493 4.66824827 4.0636104 ]
 [5.00661389 4.83041553 4.8573831  ... 0.79615493 4.66824827 4.0636104 ]
 ...
 [5.00661389 4.83041553 4.8573831  ... 0.79615493 4.66824827 4.0636104 ]
 [5.00661389 4.83041553 4.8573831  ... 0.79615493 4.66824827 4.0636104 ]
 [5.00661389 4.83041553 4.8573831  ... 0.79615493 4.66824827 4.0636104 ]]
26244
Epoch 1/300
1000/1000 [==============================] - 1s 873us/step - loss: 50.7841
Epoch 2/300
1000/1000 [==============================] - 1s 518us/step - loss: 50.7825
Epoch 3/300
1000/1000 [==============================] - 1s 543us/step - loss: 50.7810
Epoch 4/300
1000/1000 [==============================] - 1s 587us/step - loss: 50.7795
Epoch 5/300
1000/1000 [==============================] - 1s 578us/step - loss: 50.7779
Epoch 6/300
1000/1000 [==============================] - 1s 530us/step - loss: 50.7764
Epoch 7/300
1000/10

Epoch 89/300
1000/1000 [==============================] - 1s 649us/step - loss: 50.6480
Epoch 90/300
1000/1000 [==============================] - 1s 679us/step - loss: 50.6465
Epoch 91/300
1000/1000 [==============================] - 1s 559us/step - loss: 50.6450
Epoch 92/300
1000/1000 [==============================] - 1s 591us/step - loss: 50.6434
Epoch 93/300
1000/1000 [==============================] - 1s 635us/step - loss: 50.6419
Epoch 94/300
1000/1000 [==============================] - 1s 696us/step - loss: 50.6403
Epoch 95/300
1000/1000 [==============================] - 1s 509us/step - loss: 50.6388
Epoch 96/300
1000/1000 [==============================] - 1s 556us/step - loss: 50.6372
Epoch 97/300
1000/1000 [==============================] - 1s 599us/step - loss: 50.6357
Epoch 98/300
1000/1000 [==============================] - 1s 597us/step - loss: 50.6342
Epoch 99/300
1000/1000 [==============================] - 1s 547us/step - loss: 50.6326
Epoch 100/300
1000/1000 [=======

Epoch 181/300
1000/1000 [==============================] - 1s 609us/step - loss: 50.5062
Epoch 182/300
1000/1000 [==============================] - 1s 557us/step - loss: 50.5047
Epoch 183/300
1000/1000 [==============================] - 1s 596us/step - loss: 50.5031
Epoch 184/300
1000/1000 [==============================] - 1s 668us/step - loss: 50.5016
Epoch 185/300
1000/1000 [==============================] - 1s 711us/step - loss: 50.5000
Epoch 186/300
1000/1000 [==============================] - 1s 676us/step - loss: 50.4985
Epoch 187/300
1000/1000 [==============================] - 1s 586us/step - loss: 50.4970
Epoch 188/300
1000/1000 [==============================] - 1s 663us/step - loss: 50.4954
Epoch 189/300
1000/1000 [==============================] - 1s 584us/step - loss: 50.4939
Epoch 190/300
1000/1000 [==============================] - 1s 560us/step - loss: 50.4924
Epoch 191/300
1000/1000 [==============================] - 1s 552us/step - loss: 50.4908
Epoch 192/300
1000/10

In [2]:
graph = test_build_relation_graph_with_symertic_data()
data = test_get_matix_for_autoencoder(graph)
test_autoencoder(data)

-------------RelationGraph-------------
Gene	1219
3	ann-(1219, 116), expr-(1219, 282), ppi-(1219, 1219)
2	ann_T-(116, 1219), expr_T-(282, 1219)
GO term	116
1	ann_T-(116, 1219)
1	ann-(1219, 116)
Experimental condition	282
1	expr_T-(282, 1219)
1	expr-(1219, 282)

ann (1219, 116)
expr (1219, 282)
ppi (1219, 1219)
ann_T (116, 1219)
expr_T (282, 1219)
Gene: 122
GO term: 12
Experimental condition: 28
[[5.00661389 4.83041553 4.8573831  ... 0.79615493 4.66824827 4.0636104 ]
 [5.00661389 4.83041553 4.8573831  ... 0.79615493 4.66824827 4.0636104 ]
 [5.00661389 4.83041553 4.8573831  ... 0.79615493 4.66824827 4.0636104 ]
 ...
 [5.00661389 4.83041553 4.8573831  ... 0.79615493 4.66824827 4.0636104 ]
 [5.00661389 4.83041553 4.8573831  ... 0.79615493 4.66824827 4.0636104 ]
 [5.00661389 4.83041553 4.8573831  ... 0.79615493 4.66824827 4.0636104 ]]
26244
Epoch 1/300
100/100 [==============================] - 2s 16ms/step - loss: 50.7874
Epoch 2/300
100/100 [==============================] - 0s 566us/step

100/100 [==============================] - 0s 636us/step - loss: 50.7571
Epoch 181/300
100/100 [==============================] - 0s 616us/step - loss: 50.7569
Epoch 182/300
100/100 [==============================] - 0s 540us/step - loss: 50.7568
Epoch 183/300
100/100 [==============================] - 0s 659us/step - loss: 50.7566
Epoch 184/300
100/100 [==============================] - 0s 554us/step - loss: 50.7565
Epoch 185/300
100/100 [==============================] - 0s 623us/step - loss: 50.7563
Epoch 186/300
100/100 [==============================] - 0s 674us/step - loss: 50.7562
Epoch 187/300
100/100 [==============================] - 0s 794us/step - loss: 50.7560
Epoch 188/300
100/100 [==============================] - 0s 766us/step - loss: 50.7558
Epoch 189/300
100/100 [==============================] - 0s 679us/step - loss: 50.7557
Epoch 190/300
100/100 [==============================] - 0s 772us/step - loss: 50.7555
Epoch 191/300
100/100 [==============================] - 

100/100 [==============================] - 0s 561us/step - loss: 50.7425
Epoch 275/300
100/100 [==============================] - 0s 697us/step - loss: 50.7424
Epoch 276/300
100/100 [==============================] - 0s 557us/step - loss: 50.7422
Epoch 277/300
100/100 [==============================] - 0s 556us/step - loss: 50.7421
Epoch 278/300
100/100 [==============================] - 0s 479us/step - loss: 50.7419
Epoch 279/300
100/100 [==============================] - 0s 608us/step - loss: 50.7418
Epoch 280/300
100/100 [==============================] - 0s 506us/step - loss: 50.7416
Epoch 281/300
100/100 [==============================] - 0s 514us/step - loss: 50.7415
Epoch 282/300
100/100 [==============================] - 0s 497us/step - loss: 50.7413
Epoch 283/300
100/100 [==============================] - 0s 456us/step - loss: 50.7412
Epoch 284/300
100/100 [==============================] - 0s 558us/step - loss: 50.7410
Epoch 285/300
100/100 [==============================] - 

ValueError: Error when checking input: expected input_1 to have shape (26244,) but got array with shape (162,)